# Black Litterman Model

In [331]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [332]:
#Recebe DF com preços e devolve retorno médio anual
def in_return_cov(df): 
    price = df.copy()
    price = (price / price.shift(1)) -1
    price.iloc[0,:] = 0 # Primeira linha = 0     
    
    returns = np.matrix(price)
    mean_returns = np.mean(returns, axis = 0)
    
    annual_returns = np.array([])
    for i in range(len(np.transpose(mean_returns))):
        annual_returns = np.append(annual_returns,(mean_returns[0,i]+1)**252-1)   
    
    cov = price.cov()*252
    
    return (annual_returns, np.matrix(cov))

#Carrega tabela de preços
def load_prices(symbol, index, start_date, end_date):
    
    dates = pd.date_range(start_date, end_date)
    dfBase = pd.DataFrame(index = dates)
    
    dfIndex = pd.read_csv("C:/Users/Guilherme Overney/Desktop/DATA/{}.csv".format(index), index_col = "Date", parse_dates = True,
                     usecols = ['Date', 'Adj Close'], na_values=['nan'])
    dfIndex = dfIndex.dropna()
    dfIndex = dfIndex.rename(columns={'Adj Close': '{}'.format(index)})
    dfBase = dfBase.join(dfIndex, how = 'inner')
    
    
    for symbol in symbols:
        df_temp = pd.read_csv("C:/Users/Guilherme Overney/Desktop/DATA/{}.csv".format(symbol), index_col = "Date", parse_dates = True, 
                     usecols = ['Date', 'Adj Close'], na_values=['nan'])
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        dfBase = dfBase.join(df_temp)
    
    dfBase.fillna(method = "ffill", inplace = True) #Tratando dados 
    dfBase.fillna(method = "bfill", inplace = True)
    
    return dfBase.drop('IBOV',axis = 1) #Tiro o índice da tabela de preços...

#Calcula o retorno de um portfólio de ativos
def port_return(W,r):
    return sum(W*r)

#Calcula o peso inicial do portfolio dado o tamanho de mercado do papel
def mkt_weights(weights):
    return np.array(weights) / sum(weights)

def risk_return(W,S,r,rf=0.06):
    var = np.dot(np.dot(W,S),np.transpose(W))
    port_r = port_return(W,r)
    return np.asscalar((port_r - rf) / var)

#Calcula a matriz de retorno implicito
def vector_equilibrium_return(S, W, r ):
    # rf = risk free rate
    # S = Covarianve matrix
    # w = weights
    
    A = risk_return(W,S,r)
    
    return (np.dot(A,np.dot(S,W)))

def diago_omega(t, P, S):
    # t = tau - Medida de incerteza da estimativa dos retornos históricos
    # S = Covarianve matrix
    # P = Link matrix
    
    omega = np.dot(t,np.dot(P,np.dot(S,np.transpose(P))))
    
    for i in range(len(omega)):
        for y in range(len(omega)):
            if i != y: omega[i,y] = 0
    return omega
    
#Black Litterman Model
def posterior_estimate_return(t,S,P,Q,PI):
    # t = tau - Medida de incerteza da estimativa dos retornos históricos
    # S = Covarianve matrix
    # P = Link matrix
    # Q = View matrix
    # PI = Return Matrix
    
    omega = diago_omega(t, P, S)
    
    parte_1 = t*np.dot(S,np.transpose(P))
    parte_2 = np.linalg.inv(np.dot(P*t,np.dot(S,np.transpose(P))) + omega)
    parte_3 = Q - np.dot(P,np.transpose(PI))
    
    return np.transpose(PI) + np.dot(parte_1,np.dot(parte_2,parte_3))

def posterior_covariance(t,S,P,PI):
    # t = tau - Medida de incerteza da estimativa dos retornos históricos
    # S = Covarianve matrix
    # P = Link matrix
    # PI = Return Matrix
    
    omega = diago_omega(t, P, S)
    
    parte_1 = t*np.dot(S,np.transpose(P))
    parte_2 = np.linalg.inv(t*np.dot(P,np.dot(S,np.transpose(P)))+omega)
    parte_3 = t*np.dot(P,S)

    return t*S - np.dot(parte_1,np.dot(parte_2,parte_3))


# Iniciando o modelo...

In [333]:
# Range de trabalho
start_date = '2006-01-01'
end_date = '2018-12-31'

In [334]:
symbols = ['ABEV3','BBAS3','GGBR4','ITSA4','ITUB4','PETR4','VALE3']
#Tamanho de mercado dos papéis
weights = [278.562, 137.299, 23.69, 116.46, 333.782, 350.272, 270.673]

In [335]:
precos = load_prices(symbols, 'IBOV', start_date, end_date)

In [336]:
assets_return , S= in_return_cov(precos)

# Covariance Matrix

In [337]:
display(pd.DataFrame(S,columns = symbols,index = symbols))

,ABEV3,BBAS3,GGBR4,ITSA4,ITUB4,PETR4,VALE3
ABEV3,0.085025,0.041408,0.039632,0.036378,0.040462,0.035945,0.041387
BBAS3,0.041408,0.191795,0.090565,0.102348,0.109948,0.107081,0.082307
GGBR4,0.039632,0.090565,0.204286,0.082474,0.082199,0.107405,0.125590
ITSA4,0.036378,0.102348,0.082474,0.251237,0.113210,0.087396,0.071458
ITUB4,0.040462,0.109948,0.082199,0.113210,0.130627,0.086840,0.078441
PETR4,0.035945,0.107081,0.107405,0.087396,0.086840,0.202475,0.102757
VALE3,0.041387,0.082307,0.125590,0.071458,0.078441,0.102757,0.187783


In [338]:
W = mkt_weights(weights)

In [339]:
display(pd.DataFrame({'Annual Mean Return %': assets_return*100.00, 'Weight (based on market cap)': W}, index=symbols).T)

,ABEV3,BBAS3,GGBR4,ITSA4,ITUB4,PETR4,VALE3
Annual Mean Return %,35.214366,25.984275,14.825898,27.825301,21.132961,14.726054,20.324718
Weight (based on market cap),0.184388,0.090882,0.015681,0.077088,0.220940,0.231855,0.179166


# Delta - Coeficiente de aversão ao risco

In [340]:
A = risk_return(W,S,assets_return)
display(A)

1.9006341000153895

### Vetor de equilíbrio dos retornos
Calculado usando otimização reversa.

In [341]:
PI = vector_equilibrium_return(S, W, assets_return)
display(pd.DataFrame(np.transpose(PI), index=symbols))

,0
ABEV3,0.090385
BBAS3,0.186722
GGBR4,0.172320
ITSA4,0.180082
ITUB4,0.172042
PETR4,0.207783
VALE3,0.185101


In [342]:
Q = np.zeros((3,1))
Q[0,0] = 0.005
Q[1,0] = 0.01
Q[2,0] = 0.005

P = np.zeros((3,7))
P[0,5] = 1
P[0,6] = -1
P[1,2] = 1
P[1,0] = -1
P[2,1] = 1
P[2,3] = -1


display(pd.DataFrame(Q, columns = ["Views"]) )

display(pd.DataFrame(P,columns = symbols) )

,Views
0,0.005
1,0.010
2,0.005


,ABEV3,BBAS3,GGBR4,ITSA4,ITUB4,PETR4,VALE3
0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0
1,-1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,-1.0,0.0,0.0,0.0


## Calculando a estimativa posterior dos retornos

In [343]:
t = 1 / len(precos) # Medida de incerteza das estimativas de retornos pelo método Meucci
expc_return = posterior_estimate_return(t,S,P,Q,PI)
display(pd.DataFrame(expc_return, index = symbols))

,0
ABEV3,0.098518
BBAS3,0.176773
GGBR4,0.144826
ITSA4,0.171455
ITUB4,0.164380
PETR4,0.190041
VALE3,0.175105


## Calculando a variância posterior

In [344]:
post_cov = posterior_covariance(t,S,P,PI)
display(pd.DataFrame(post_cov, index = symbols, columns = symbols))

,ABEV3,BBAS3,GGBR4,ITSA4,ITUB4,PETR4,VALE3
ABEV3,0.000025,0.000014,0.000018,0.000014,0.000014,0.000014,0.000015
BBAS3,0.000014,0.000052,0.000022,0.000038,0.000033,0.000028,0.000024
GGBR4,0.000018,0.000022,0.000043,0.000021,0.000021,0.000026,0.000028
ITSA4,0.000014,0.000038,0.000021,0.000062,0.000033,0.000025,0.000022
ITUB4,0.000014,0.000033,0.000021,0.000033,0.000039,0.000024,0.000023
PETR4,0.000014,0.000028,0.000026,0.000025,0.000024,0.000050,0.000034
VALE3,0.000015,0.000024,0.000028,0.000022,0.000023,0.000034,0.000047


##  Apurando covariância dos retornos sobre o retorno médio estimado.

In [345]:
cov_post_estimate = post_cov + S

display(pd.DataFrame(cov_post_estimate, index = symbols, columns = symbols))

,ABEV3,BBAS3,GGBR4,ITSA4,ITUB4,PETR4,VALE3
ABEV3,0.085050,0.041422,0.039650,0.036391,0.040476,0.035959,0.041402
BBAS3,0.041422,0.191848,0.090587,0.102387,0.109981,0.107109,0.082331
GGBR4,0.039650,0.090587,0.204330,0.082495,0.082220,0.107430,0.125617
ITSA4,0.036391,0.102387,0.082495,0.251298,0.113244,0.087421,0.071480
ITUB4,0.040476,0.109981,0.082220,0.113244,0.130667,0.086864,0.078463
PETR4,0.035959,0.107109,0.107430,0.087421,0.086864,0.202526,0.102792
VALE3,0.041402,0.082331,0.125617,0.071480,0.078463,0.102792,0.187831


## Novo peso do portfolio

In [346]:
new_weigth = np.dot(np.transpose(expc_return),np.linalg.inv(A*cov_post_estimate))
display(pd.DataFrame(new_weigth, columns = symbols).T)

,0
ABEV3,0.275231
BBAS3,0.090155
GGBR4,-0.075224
ITSA4,0.077763
ITUB4,0.220871
PETR4,0.203501
VALE3,0.207392
